In [ ]:
import pandas as pd
import scipy
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE

In [ ]:
# for testing modularized functions
df_path = "" # edit this to select dataset
df = pd.read_csv(df_path)

In [ ]:
def remove_columns(df: pd.DataFrame):
    '''
    Removes transcript_name and gene_id columns.
    Add or remove columns as required.
    '''
    df1 = df.drop(["transcript_name", "gene_id"], axis=1)
    return df1

# TODO: for trigram tokenization
def trigram_tokenize(df:pd.DataFrame, n, corpus):
    '''
    to apply trigram on column "nucleotide_seq" using TfidfVectorizer
    '''
    #corpus = df['nucleotide_seq']
    vectorizer = TfidfVectorizer(analyzer='char', ngram_range=(n,n)).fit(corpus)

    v_nucleotide_seq = vectorizer.transform(df["nucleotide_seq"]) 
    v = pd.DataFrame(scipy.sparse.csr_matrix.toarray(v_nucleotide_seq)) # convert sparse matrix to array

    # creating dictionary to easily add vectorized sequence features as columns to dataframe
    new_nucleotide_data = dict() 
    for i in range(v.shape[1]):
        key = "s" + str(i)
        new_nucleotide_data[key] = v[i]

    df_final = df.assign(**new_nucleotide_data)
    
    return(df_final) #returns dataframe with vectorized nucleotide features as columns

def data_split(df:pd.DataFrame, test_prop=0.2):
    '''
    splits data into train and test sets according to the percentage given.
    '''
    X = df.drop(["label"], axis=1)
    y = df["label"]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
    return X_train, X_test, y_train, y_test

def standardize_data(X_train, X_test, y_train, y_test):
    '''
    Standardizes numerical features while leaving non-numerical features unchanged.
    Assumes the input X_train and X_test are pandas DataFrames.
    '''
    
    # Identify numerical and categorical columns
    numerical_cols = X_train.select_dtypes(include=['int64', 'float64']).columns
    categorical_cols = X_train.select_dtypes(exclude=['int64', 'float64']).columns

    preprocessor = ColumnTransformer(
        transformers=[
            ('num', StandardScaler(), numerical_cols),
            ('cat', 'passthrough', categorical_cols)  # Leave categorical columns unchanged
        ])
    
    # Apply the column transformer
    X_train_scaled = preprocessor.fit_transform(X_train)
    X_test_scaled = preprocessor.transform(X_test)
    
    return X_train_scaled, X_test_scaled, y_train, y_test

def synthetic_oversampling(X_train, y_train):
    '''
    Uses SMOTE to oversample the minority class.
    '''
    smote = SMOTE(random_state=42)
    X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)
    return X_train_resampled, y_train_resampled
